In [90]:
from functools import partial

import numpy as np
from tqdm.notebook import tqdm

In [91]:
def interactions_harmonic(q, k=1.0):
    """Evaluate potential energy and its derivatives at configuration `x` for a harmonic potential."""
    U = k * 0.5 * (q**2).sum()
    dU_dq = k * q
    return U, dU_dq

In [92]:
def interactions_double_well(q, D0=1.0, a=1.0, l=1.0):
    """Evaluate potential energy and its derivatives at configuration `x` for a double well potential."""
    U = ((D0 / a**4) * ((q**2 - a**2)**2) + l*q).sum()
    dU_dq = 4 * (D0 / a**4) * (q**2 - a**2) * q + l
    return U, dU_dq

In [93]:
def do_output():

    # Note that this relies on global variables. That is not ideal, but will do for this purpose.

    # calculate kinetic energy and temperature
    E_kin = 0.5 * (p**2 / m_DOF).sum()
    T_kin = 2 * E_kin / (k_B * 3 * N)

    # prepate format string for one energy file line
    fmt_energy = '{:6d} {:10.3f} {:12.6f} {:12.6f} {:12.6f} {:12.6f}\n'

    fs_out['energy'].write(fmt_energy.format(i, t, E_kin, E_pot, E_kin+E_pot, T_kin))

    # comment line for XYZ files
    comment = 'step {:d}'.format(i)

    # write XYZ files - positions and velocities
    write_xyz_frame(fs_out['position'], q, names, comment=comment)
    write_xyz_frame(fs_out['velocity'], p/m_DOF, names, comment=comment)

In [94]:
def write_xyz_frame(f_out, q, names, comment=''):
    """Write one XYZ frame to an open file."""

    N = q.shape[0]

    # number of atoms
    f_out.write('{:d}\n'.format(N))

    # comment line
    f_out.write(comment + '\n')

    # one line per atom
    for i in range(N):
        data = names[i], q[i, 0], q[i, 1], q[i, 2]
        f_out.write('{:3s} {:12.6f} {:12.6f} {:12.6f}\n'.format(*data))

## Simulate

An arbitrary 1D potential, simulated simultaneously for 3xN independent degrees of freedom.

In [95]:
#
# settings
#

# propagation
n_steps = 50000            # number of MD steps
dt = 0.2                    # time step

# atomic properties
N = 5                      # number of atoms
names = ('X',) * N          # names of all atoms
masses = {                  # masses of all atom types
    'X': 1.5
}

# thermostat
k_B = 1.0                   # Boltzmann constant
T_init = 1.6                # temperature of initial distribution
T = 1.0                   # temperature of the thermostat
tau = 250                   # thermostat time constant
p_resample = dt / tau

# output
stride_out = 10             # period of file output
fn_energy = 'energy.txt'
fn_positions = 'positions.xyz'
fn_velocities = 'velocities.xyz'

# pick an interaction potential
interactions = partial(interactions_double_well, l=0)

In [96]:
#
# initialize everything
#

# simulation time starts at 0
t = 0.0

# prepare masses, their broadcasting version, and scale of thermal momenta
m = np.array([masses[name] for name in names], dtype=float)
m_DOF = m[:, np.newaxis]
scale_p = np.sqrt(k_B * T * np.repeat(m_DOF, 3, axis=1))
scale_p_init = np.sqrt(k_B * T_init * np.repeat(m_DOF, 3, axis=1))

# initial conditions - positions
q = np.zeros((N, 3))

# initial conditions - constant or thermal momenta
#p = np.ones_like(q)
p = np.random.normal(0, scale_p_init)

# prepare thermostat
gamma = 2 / tau
A = np.exp(-gamma * dt)
B = np.sqrt((1-A**2) * k_B * T * m_DOF)

# update interactions for initial positions
E_pot, dU_dq = interactions(q)
F = - dU_dq

# open output files
fs_out = {
    'position': open(fn_positions, 'w'),
    'velocity': open(fn_velocities, 'w'),
    'energy': open(fn_energy, 'w'),
}

In [97]:
#
# run the simulation
#

print('SimpleMD')
print('========')
print()

# main loop
for i in tqdm(range(n_steps)):

    # write output
    if i % stride_out == 0:
        do_output()

    # propagator: start

    # propagate momenta for half a step
    p += F * dt / 2

    # propagate positions for a half a step
    q += dt * p / (m_DOF * 2)

    # apply Langevin thermostat to momenta, full step
    """p *= A
    p += B * np.random.normal(0, 1, size=p.shape)"""

    # propagate positions for a half a step
    q += dt * p / (m_DOF * 2)

    # full position update done, update potential energy and accelerations
    E_pot, dU_dq = interactions(q)
    F[:, :] = - dU_dq

    # propagate momenta for half a step
    p += F * dt / 2

    # Andersen thermostat - resample some of the velocities
    idx = np.where(np.random.random(N) < p_resample) 
    p[idx] = np.random.normal(0, scale_p[idx])

    # propagator: end

    # update time
    t += dt

# final step
i += 1

# write output for last frame
do_output()

# close output files
for f_out in fs_out.values():
    f_out.close()

print('SimpleMD finished.')

SimpleMD



  0%|          | 0/50000 [00:00<?, ?it/s]

SimpleMD finished.
